In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
import random
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings("ignore")

# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# algoritmos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# algoritmos de regresión
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# métricas
import sklearn.metrics as metrics

# hiperparametrización
from sklearn.model_selection import GridSearchCV

# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

import xgboost as xgb

from numpy import random
random.seed(42)

from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ruta_archivo = "C:/Users/Herre/Desktop/TFM/Nuclio_EasyMoney/0_Data/df_encoding.csv"

# Leer el archivo
df_clustering = pd.read_csv(ruta_archivo)

A más información tengamos sobre los clientes mejor será la segmentación que hagamos, es por eso que hemos decidido crear nuevas variables a partir de las columnas ya existentes.

# Crear Nuevas Features

El objetivo de crear nuevas features es para poder hacer una mejor segmentación de los clientes.

**1.-number_of_products:** Número de productos financieros que el cliente tiene con el banco (si tienes una columna que lo indique).

Esta columna podría representar el número total de productos financieros que un cliente tiene con el banco. A partir de las columnas disponibles, podemos considerar cualquier columna que indique un producto financiero (por ejemplo, depósitos, préstamos, tarjetas de crédito, etc.) como parte del total de productos.

**2.-account_balance**: Saldo promedio de la cuenta, si está disponible, para indicar la capacidad financiera.

Aunque no tenemos una columna específica para account_balance, podríamos inferir un valor basado en los activos del cliente, como depósitos a corto plazo (short_term_deposit), fondos, valores, etc., si esos valores son cuantitativos. Si estas columnas indican si el cliente tiene estos productos (con valores binarios 0 o 1), no podremos calcular el balance. Necesitamos clarificar qué representa.

**3.-loan_amount o debt_amount:** Si tienes columnas relacionadas con la deuda o préstamos, estas pueden ser útiles para segmentar clientes según el riesgo financiero. 

Esta columna puede representar el monto total del préstamo de un cliente. Podemos inferirlo de columnas relacionadas con productos de préstamos como loans y mortgage (si esos valores representan montos y no solo una indicación de presencia del producto).

**4.-product_engagement_score (Puntuación de compromiso con productos):** Esta feature mide el nivel de compromiso de un cliente con los productos financieros del banco. Calcula una puntuación basada en el número y el tipo de productos que un cliente posee. Algunos productos pueden tener mayor peso en la puntuación si son indicadores de un mayor compromiso o valor del cliente, por ejemplo, tener una cuenta de pensiones puede indicar una relación más duradera y comprometida con el banco.

Este puntaje puede ayudar a diferenciar clientes altamente comprometidos que utilizan múltiples productos y servicios financieros, de aquellos que tienen una relación más transaccional o limitada con el banco. Los clientes con un alto product_engagement_score pueden pertenecer a clusters que merecen una atención especial en términos de retención o up-selling.

product_engagement_score = number_of_products + 2 * (credit_card + debit_card) + 3 * (pension_plan + emc_account + em_account_pp + em_account_p + em_acount)
Aquí, las tarjetas de crédito y débito tienen un peso extra (multiplicado por 2), y los productos de inversión a largo plazo o de mayor valor tienen un peso aún mayor (multiplicado por 3).

**5.-debt_to_income_ratio (Relación deuda-ingresos):** Esta feature calcula la proporción entre la deuda total del cliente (suma de préstamos y hipotecas) y su ingreso (salary). Es un indicador clásico en la evaluación de la salud financiera de un cliente y su capacidad para asumir más deuda.

Esta métrica es crucial para identificar clusters de clientes con diferentes perfiles de riesgo financiero. Los clientes con una alta relación deuda-ingreso pueden ser considerados de alto riesgo y, por tanto, se agruparían en clusters que requieren un enfoque más cauteloso o restricciones crediticias.

debt_to_income_ratio = (loans + mortgage) / salary
Esta fórmula asume que todas las cantidades están en las mismas unidades y que no hay valores nulos en las columnas utilizadas. Podrías agregar un manejo de excepciones para evitar divisiones por cero o datos faltantes.

In [3]:
# 1. Crear 'number_of_products'
# Suponiendo que cada columna de productos (como 'short_term_deposit', 'loans', etc.) tiene un valor de 0 o 1
product_columns = ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 
                   'em_account_pp', 'credit_card', 'payroll', 'pension_plan', 'payroll_account', 
                   'emc_account', 'debit_card', 'em_account_p', 'em_acount']

df_clustering['number_of_products'] = df_clustering[product_columns].sum(axis=1)

# 2. Crear 'account_balance'
#Esto lo que hace es sumar los valores binarios de cada columna señalada.
balance_columns = ['short_term_deposit', 'funds', 'securities', 'long_term_deposit']

df_clustering['account_balance'] = df_clustering[balance_columns].sum(axis=1)

In [4]:
df_clustering['number_of_products'].value_counts()

number_of_products
1.0    266337
0.0    116068
2.0     38570
3.0     11460
4.0      8455
5.0      3337
6.0      1036
7.0       193
8.0        31
9.0         2
Name: count, dtype: int64

In [5]:
#3.-loan_amount o debt_amount:
# Crear la nueva columna 'loan_amount' según las condiciones especificadas
df_clustering['loan_amount'] = 0  # Inicialmente, todos se consideran sin deudas (0)

# Si tienen un 'loan' o 'mortgage', pero no ambos, se asigna un 1
df_clustering.loc[(df_clustering['loans'] == 1) & (df_clustering['mortgage'] == 0), 'loan_amount'] = 1
df_clustering.loc[(df_clustering['loans'] == 0) & (df_clustering['mortgage'] == 1), 'loan_amount'] = 1

# Si tienen ambos 'loan' y 'mortgage', se asigna un 2
df_clustering.loc[(df_clustering['loans'] == 1) & (df_clustering['mortgage'] == 1), 'loan_amount'] = 2

# Verifica la columna creada
print(df_clustering[['loans', 'mortgage', 'loan_amount']].head())

   loans  mortgage  loan_amount
0      0         0            0
1      0         0            0
2      0         0            0
3      0         0            0
4      0         0            0


In [6]:
# 4.-product_engagement_score (Puntuación de compromiso con productos):
# Lista de columnas de productos con diferentes niveles de compromiso
high_commitment_products = ['pension_plan', 'emc_account', 'em_account_pp', 'em_account_p', 'em_acount']  # Productos de alto compromiso
medium_commitment_products = ['credit_card', 'debit_card']  # Productos de compromiso medio
low_commitment_products = ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 
                           'payroll', 'payroll_account']  # Productos de bajo compromiso

# Crear 'product_engagement_score'
df_clustering['product_engagement_score'] = (
    df_clustering[high_commitment_products].sum(axis=1) * 3 +   # Productos de alto compromiso, peso 3
    df_clustering[medium_commitment_products].sum(axis=1) * 2 +  # Productos de compromiso medio, peso 2
    df_clustering[low_commitment_products].sum(axis=1)          # Productos de bajo compromiso, peso 1
)

# 5.-debt_to_income_ratio (Relación deuda-ingresos):
# Columnas utilizadas para calcular la deuda total
debt_columns = ['loans', 'mortgage']

# Crear la columna 'debt_to_income_ratio'
# Evitamos la división por cero utilizando 'np.where' para manejar ingresos nulos o cero.
df_clustering['debt_to_income_ratio'] = df_clustering[debt_columns].sum(axis=1) / df_clustering['salary'].replace(0, np.nan)

# Reemplazar posibles NaN resultantes de la división por cero con un valor específico o dejar como NaN
df_clustering['debt_to_income_ratio'].fillna(0, inplace=True)

Guardamos el dataframe antes de crear una nueva columna con el Cluster que corresponde a cada cliente. Este dataframe nos servirá para que después podamos agrupar los productos en 3 grupos.

In [7]:
df_agrupacion = df_clustering.copy()

# Tabla Clustering_2

In [9]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clustering)
# Perform K-Means clustering
kmeans = KMeans(n_clusters=7, random_state=42)
df_clustering['cluster'] = kmeans.fit_predict(X_scaled)

In [10]:
# Definir las columnas relevantes para el clustering
summary_columns = ["salary", "loan_amount", "debt_to_income_ratio", "account_balance", "number_of_products", "product_engagement_score",
"age", "segment", "active_customer", "region_code"]

# Inicializar el DataFrame ficha_df
ficha_df = pd.DataFrame()

# Generar los datos resumen para cada columna
for col in summary_columns:
    resumen_data = df_clustering[["cluster", col]].groupby("cluster").describe().T[1:]
    ficha_df = pd.concat([ficha_df, resumen_data])

# Definir los componentes del multi-índice
out_index = ["Monetario"] *4  + ["Productos"] * 2 + ["Social"] * 4  # Ajusta esta lista según las columnas seleccionadas
inner_index = summary_columns
estadisticos = ["mean", "std", "min", "25%", "50%", "75%", "max"]

# Crear el nuevo multi-índice
new_multi_index = []
for oi, ii in zip(out_index, inner_index):
    for es in estadisticos:
        new_multi_index.append((oi, ii, es))

# Generar el multi-índice
def generate_multiindex(list_of_tuples, names):
    return pd.MultiIndex.from_tuples(list_of_tuples, names=names)

names = ["Grupo Indicadores", "Indicador", "Estadístico"]
index_ficha = generate_multiindex(new_multi_index, names)

# Establecer el multi-índice en ficha_df
ficha_df.set_index(index_ficha, inplace=True)

# Calcular el tamaño de cada cluster
tamaño_clusters = df_clustering.groupby("cluster").size().to_frame().T
tamaño_clusters.set_index(generate_multiindex([("General", "Clúster", "Tamaño")], names), inplace=True)

# Concatenar los datos de tamaño con el DataFrame ficha_df
ficha_df = pd.concat([tamaño_clusters, ficha_df])

# Estilizar el DataFrame con un gradiente de fondo
styled_ficha_df = ficha_df.style.background_gradient(cmap='Blues', axis=1)

# Mostrar el DataFrame estilizado
styled_ficha_df

In [12]:
df_clustering.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445489 entries, 0 to 445488
Data columns (total 42 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   pk_cid                    445489 non-null  int64  
 1   active_customer           445489 non-null  float64
 2   segment                   445489 non-null  int64  
 3   short_term_deposit        445489 non-null  int64  
 4   loans                     445489 non-null  int64  
 5   mortgage                  445489 non-null  int64  
 6   funds                     445489 non-null  int64  
 7   securities                445489 non-null  int64  
 8   long_term_deposit         445489 non-null  int64  
 9   em_account_pp             445489 non-null  int64  
 10  credit_card               445489 non-null  int64  
 11  payroll                   445489 non-null  float64
 12  pension_plan              445489 non-null  float64
 13  payroll_account           445489 non-null  i

# Medias generales de los Clusters

Queremos saber la inforamción relevante de cada cluster

#### Cluster 0

In [34]:
cluster_0_salary = df_clustering[df_clustering['cluster'] == 0]['salary']
cluster_0_salary.sum() / 85258

112740.7394526027

In [35]:
cluster_0_age = df_clustering[df_clustering['cluster'] == 0]['age']
cluster_0_age.sum() / 85258

41.74042318609397

In [36]:
cluster_0_number_of_products = df_clustering[df_clustering['cluster'] == 0]['number_of_products']
cluster_0_number_of_products.sum() / 85258

1.0828426657908936

#### Cluster 1

In [40]:
cluster_1_salary = df_clustering[df_clustering['cluster'] == 1]['salary']
cluster_1_salary.sum() / 160850

108166.46750805096

In [38]:
cluster_1_age = df_clustering[df_clustering['cluster'] == 1]['age']
cluster_1_age.sum() / 160850

25.315629468448865

In [41]:
cluster_1_number_of_products = df_clustering[df_clustering['cluster'] == 1]['number_of_products']
cluster_1_number_of_products.sum() / 160850

1.032589368977308

#### Cluster 2

In [42]:
cluster_2_salary = df_clustering[df_clustering['cluster'] == 2]['salary']
cluster_2_salary.sum() / 5565

137234.06591374663

In [43]:
cluster_2_age = df_clustering[df_clustering['cluster'] == 2]['age']
cluster_2_age.sum() / 5565

34.71482479784367

In [44]:
cluster_2_number_of_products = df_clustering[df_clustering['cluster'] == 2]['number_of_products']
cluster_2_number_of_products.sum() / 5565

0.0472596585804133

#### Cluster 3

In [48]:
cluster_3_salary = df_clustering[df_clustering['cluster'] == 3]['salary']
cluster_3_salary.sum() / 75009

103192.28444386672

In [49]:
cluster_3_age = df_clustering[df_clustering['cluster'] == 3]['age']
cluster_3_age.sum() / 75009

22.479195829833753

In [50]:
cluster_3_number_of_products = df_clustering[df_clustering['cluster'] == 3]['number_of_products']
cluster_3_number_of_products.sum() / 75009

0.73727152741671

#### Cluster 4

In [51]:
cluster_4_salary = df_clustering[df_clustering['cluster'] == 4]['salary']
cluster_4_salary.sum() / 54074

117527.62146706364

In [52]:
cluster_4_age = df_clustering[df_clustering['cluster'] == 4]['age']
cluster_4_age.sum() / 54074

40.28740244849651

In [53]:
cluster_4_number_of_products = df_clustering[df_clustering['cluster'] == 4]['number_of_products']
cluster_4_number_of_products.sum() / 54074

0.9307060694603692

#### Cluster 5

In [54]:
cluster_5_salary = df_clustering[df_clustering['cluster'] == 5]['salary']
cluster_5_salary.sum() / 17495

114621.04887796512

In [55]:
cluster_5_age = df_clustering[df_clustering['cluster'] == 5]['age']
cluster_5_age.sum() / 17495

37.329008288082306

In [57]:
cluster_5_number_of_products = df_clustering[df_clustering['cluster'] == 5]['number_of_products']
cluster_5_number_of_products.sum() / 17495

4.010574449842812

#### Cluster 6

In [58]:
cluster_6_salary = df_clustering[df_clustering['cluster'] == 6]['salary']
cluster_6_salary.sum() / 47238

144992.62860726533

In [59]:
cluster_6_age = df_clustering[df_clustering['cluster'] == 6]['age']
cluster_6_age.sum() / 47238

25.14043778314069

In [60]:
cluster_6_number_of_products = df_clustering[df_clustering['cluster'] == 6]['number_of_products']
cluster_6_number_of_products.sum() / 47238

0.0365172107201829

In [61]:
df_clustering.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445489 entries, 0 to 445488
Data columns (total 42 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   pk_cid                    445489 non-null  int64  
 1   active_customer           445489 non-null  float64
 2   segment                   445489 non-null  int64  
 3   short_term_deposit        445489 non-null  int64  
 4   loans                     445489 non-null  int64  
 5   mortgage                  445489 non-null  int64  
 6   funds                     445489 non-null  int64  
 7   securities                445489 non-null  int64  
 8   long_term_deposit         445489 non-null  int64  
 9   em_account_pp             445489 non-null  int64  
 10  credit_card               445489 non-null  int64  
 11  payroll                   445489 non-null  float64
 12  pension_plan              445489 non-null  float64
 13  payroll_account           445489 non-null  i

# Exportamos CSV

In [ ]:
# Guardar resultados
df_agrupacion.to_csv('df_agrupacion.csv', index=False)

In [13]:
# Guardar resultados
df_clustering.to_csv('df_Clustering.csv', index=False)